# https://www.forecastclub.blog/2023/02/how-to-transform-time-series-for-deep.html
## https://colab.research.google.com/drive/1jRwbFe_Tr9Iw-Vk5cfQRo1RS-0pbPcqQ#scrollTo=1yHWlZX4RnV4

Pembelajaran terawasi melibatkan pelatihan model pembelajaran mesin dengan set data masukan. Set data ini biasanya berupa matriks: Struktur data dua dimensi yang terdiri dari baris (sampel) dan kolom (fitur). Deret waktu adalah urutan nilai yang diurutkan dalam waktu. Jadi, perlu diubah untuk pembelajaran terbimbing. Dalam artikel sebelumnya, kita mempelajari cara mengubah deret waktu univariat dari suatu urutan menjadi matriks. Ini dilakukan dengan jendela geser. Setiap pengamatan deret dimodelkan berdasarkan nilai terkini di masa lalu, yang juga disebut lag. Berikut ini contoh transformasi ini menggunakan urutan dari 1 hingga 10:

In [5]:
#%%capture
!pip install pmdarima 

In [19]:
import pandas as pd
import numpy as np


def time_delay_embedding(series: pd.Series, n_lags: int, horizon: int):
    """
    Time delay embedding
    Time series for supervised learning
    :param series: time series as pd.Series
    :param n_lags: number of past values to used as explanatory variables
    :param horizon: how many values to forecast
    :return: pd.DataFrame with reconstructed time series
    """
    assert isinstance(series, pd.Series)

    if series.name is None:
        name = 'Series'
    else:
        name = series.name

    n_lags_iter = list(range(n_lags, -horizon, -1))

    X = [series.shift(i) for i in n_lags_iter]
    X = pd.concat(X, axis=1).dropna()
    X.columns = [f'{name}(t-{j - 1})'
                 if j > 0 else f'{name}(t+{np.abs(j) + 1})'
                 for j in n_lags_iter]

    return X

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as mae
from pmdarima.datasets import load_sunspots
#from pmdarima.utils import time_delay_embedding  # Ensure this is correctly imported

# Loading the sunspots time series
series = load_sunspots(as_series=True)

# Taking first differences to stabilize the mean and drop NaN
series_diff = series.diff().dropna()

# Applying time delay embedding
# Using 3 lags (n_lags=3) to predict the next value (horizon=1)
ts = time_delay_embedding(series=series_diff, n_lags=3, horizon=1)

# Check the columns in the time delay embedded DataFrame
print("Columns in the time series DataFrame:", ts.columns)

# Adjust target variable access based on the actual columns
# Typically, 't+1' will refer to the column that represents the next value
# Let's check the first few rows of ts to find the correct target
print(ts.head())

# Identify the target column dynamically
target_col = ts.columns[-1]  # The last column should be the target if using horizon=1
X = ts.iloc[:, :-1]  # All columns except the last one
y = ts[target_col]   # Last column as target

# Train/test split (no shuffling for time series)
X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.33, shuffle=False)

# Fitting a random forest model
model = RandomForestRegressor()
model.fit(X_tr, y_tr)

# Getting forecasts for the test set
predictions = model.predict(X_ts)

# Computing MAE error
error = mae(y_ts, predictions)
print(f'Mean Absolute Error: {error}')


Columns in the time series DataFrame: Index(['Series(t-2)', 'Series(t-1)', 'Series(t-0)', 'Series(t+1)'], dtype='object')
          Series(t-2)  Series(t-1)  Series(t-0)  Series(t+1)
May 1749          4.6          7.4        -14.3         29.3
Jun 1749          7.4        -14.3         29.3         -1.5
Jul 1749        -14.3         29.3         -1.5         11.3
Aug 1749         29.3         -1.5         11.3        -28.5
Sep 1749         -1.5         11.3        -28.5          9.6
Mean Absolute Error: 13.625049852429935


In [48]:
import re

import pandas as pd
import numpy as np


def time_delay_embedding(series: pd.Series,
                         n_lags: int,
                         horizon: int,
                         return_Xy: bool = False):
    """
    Time delay embedding
    Time series for supervised learning

    :param series: time series as pd.Series
    :param n_lags: number of past values to used as explanatory variables
    :param horizon: how many values to forecast
    :param return_Xy: whether to return the lags split from future observations

    :return: pd.DataFrame with reconstructed time series
    """
    assert isinstance(series, pd.Series)

    if series.name is None:
        name = 'Series'
    else:
        name = series.name

    n_lags_iter = list(range(n_lags, -horizon, -1))

    df_list = [series.shift(i) for i in n_lags_iter]
    df = pd.concat(df_list, axis=1).dropna()
    df.columns = [f'{name}(t-{j - 1})'
                  if j > 0 else f'{name}(t+{np.abs(j) + 1})'
                  for j in n_lags_iter]

    df.columns = [re.sub('t-0', 't', x) for x in df.columns]

    if not return_Xy:
        return df

    is_future = df.columns.str.contains('\+')

    X = df.iloc[:, ~is_future]
    Y = df.iloc[:, is_future]
    if Y.shape[1] == 1:
        Y = Y.iloc[:, 0]

    return X, Y

In [49]:
import pandas as pd

# https://github.com/vcerqueira/blog/tree/main/data
data = pd.read_csv('https://raw.githubusercontent.com/vcerqueira/blog/refs/heads/main/data/wine_sales.csv', parse_dates=['date'])
data.set_index('date', inplace=True)

series = data['Sparkling']

/tmp/ipykernel_410/3392992573.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv('https://raw.githubusercontent.com/vcerqueira/blog/refs/heads/main/data/wine_sales.csv', parse_dates=['date'])


In [62]:
# using 3 lags as explanatory variables
N_LAGS = 3
# forecasting the next 2 values
HORIZON = 2

# using a sliding window method called time delay embedding
X, Y = time_delay_embedding(series, n_lags=N_LAGS, horizon=HORIZON, return_Xy=True)

In [63]:
from sklearn.linear_model import RidgeCV

# training a ridge regression model
model = RidgeCV()
model.fit(X, Y)

RidgeCV()

In [64]:
import re
import pandas as pd
import numpy as np

def from_matrix_to_3d(df: pd.DataFrame) -> np.ndarray:
    """
    Transforming a time series from matrix into 3-d structure for deep learning
    
    :param df: (pd.DataFrame) Time series in the matrix format after embedding
    
    :return: Reshaped time series into 3-d structure 
    """    

    cols = df.columns
    
    # getting unique variables in the time series
    # this list has a single element for univariate time series
    var_names = np.unique([re.sub(r'\([^)]*\)', '', c) for c in cols]).tolist()

    # getting observation for each variable
    arr_by_var = [df.loc[:, cols.str.contains(v)].values for v in var_names]
    # reshaping the data of each variable into a 3-d format
    arr_by_var = [x.reshape(x.shape[0], x.shape[1], 1) for x in arr_by_var]
    
    # concatenating the arrays of each variable into a single array
    ts_arr = np.concatenate(arr_by_var, axis=2)

    return ts_arr


# transforming the matrices
X_3d = from_matrix_to_3d(X)
Y_3d = from_matrix_to_3d(Y)

In [65]:
#import os
#os.environ["KERAS_BACKEND"] = "torch" # 'tensorflow', 'jax´ or 'torch'
# os.environ["KERAS_BACKEND"] = "tensorflow" # 'tensorflow', 'jax´ or 'torch'
#import keras

In [66]:
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import (Dense,
                          LSTM,
                          TimeDistributed,
                          RepeatVector,Dropout)

# number of variables in the time series
# 1 because the series is univariate
N_FEATURES = 1

# creating a simple stacked LSTM
model = Sequential()
model.add(LSTM(8, activation='relu', input_shape=(N_LAGS, N_FEATURES)))
model.add(RepeatVector(HORIZON))
model.add(LSTM(4, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(N_FEATURES)))

model.compile(optimizer='adam', loss='mse')

# compiling the model
model.compile(optimizer='adam', loss='mse')

# basic train/validation split 
X_train, X_valid, Y_train, Y_valid = train_test_split(X_3d, Y_3d, test_size=.2, shuffle=False)

# training the model
model.fit(X_train, Y_train, epochs=100, validation_data=(X_valid, Y_valid))

# making predictions
preds = model.predict_on_batch(X_valid)

Epoch 1/100


/env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - loss: 7528356.0000 - val_loss: 7921101.0000
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 7976251.0000 - val_loss: 7921077.0000
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 7049875.5000 - val_loss: 7921052.5000
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 6959916.0000 - val_loss: 7921026.5000
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 7284606.0000 - val_loss: 7921002.5000
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 7586657.0000 - val_loss: 7920977.5000
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 7157991.5000 - val_loss: 7920951.5000
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 7160171.5000 - val_loss: 7920928.0000
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 8319688.0000 - val_loss: 7920903.0000
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 7125074.0000 - val_loss: 7920878.0000
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 6

# Mulyivariate

In [67]:
# transforming each variable into a matrix format
from keras.models import Sequential
from keras.layers import Dense, LSTM, TimeDistributed, RepeatVector, Dropout
from sklearn.model_selection import train_test_split

mat_by_variable = []
for col in data:
    col_df = time_delay_embedding(data[col], n_lags=N_LAGS, horizon=HORIZON)
    mat_by_variable.append(col_df)

# concatenating all variables
mat_df = pd.concat(mat_by_variable, axis=1).dropna()

# defining target (Y) and explanatory variables (X)
predictor_variables = mat_df.columns.str.contains('\(t\-|\(t\)')
target_variables = mat_df.columns.str.contains('\(t\+')
X = mat_df.iloc[:, predictor_variables]
Y = mat_df.iloc[:, target_variables]
X_3d = from_matrix_to_3d(X)
Y_3d = from_matrix_to_3d(Y)

# Define model parameters
N_LAGS = 3  # Example value, set appropriately
N_FEATURES = 1  # Example value, set appropriately
HORIZON = 1  # Example value, set appropriately

# Assuming X_3d and Y_3d are defined elsewhere
X_3d = X_3d.astype('float32')  # Ensure data type is float32
Y_3d = Y_3d.astype('float32')  # Ensure data type is float32

# Train/test split
X_train, X_valid, Y_train, Y_valid = train_test_split(X_3d, Y_3d, test_size=0.2, shuffle=False)

# Check shapes
print(f"X_train shape: {X_train.shape}")
print(f"Y_train shape: {Y_train.shape}")
print(f"X_valid shape: {X_valid.shape}")
print(f"Y_valid shape: {Y_valid.shape}")

# Initialize the model
model = Sequential()
model.add(LSTM(8, activation='relu', input_shape=(N_LAGS, N_FEATURES)))
model.add(Dropout(0.2))
model.add(RepeatVector(HORIZON))
model.add(LSTM(4, activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(N_FEATURES)))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Fit the model
model.fit(X_train, Y_train, epochs=500, validation_data=(X_valid, Y_valid))

# Make predictions
preds = model.predict(X_valid)

In [77]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, TimeDistributed, RepeatVector, Dropout
from sklearn.model_selection import train_test_split

# Define model parameters
N_LAGS = 3  # Number of lags
N_FEATURES = 1  # Number of features per time step
HORIZON = 1  # Prediction horizon

# Transform each variable into a matrix format
mat_by_variable = []
for col in data.columns:  # Make sure to iterate over column names
    col_df = time_delay_embedding(data[col], n_lags=N_LAGS, horizon=HORIZON)
    mat_by_variable.append(col_df)

# Concatenating all variables
mat_df = pd.concat(mat_by_variable, axis=1).dropna()

# Defining target (Y) and explanatory variables (X)
predictor_variables = mat_df.columns.str.contains(r'\(t\-\)|\(t\)')  # Match '(t-)' or '(t)'
target_variables = mat_df.columns.str.contains(r'\(t\+\)')  # Match '(t+)'

X = mat_df.iloc[:, predictor_variables].values  # Convert to numpy array
Y = mat_df.iloc[:, target_variables].values  # Convert to numpy array

# Function to convert 2D matrix to 3D array
def from_matrix_to_3d(matrix):
    return matrix.reshape(matrix.shape[0], N_LAGS, N_FEATURES)

# Convert matrices to 3D
X_3d = from_matrix_to_3d(X)
Y_3d = from_matrix_to_3d(Y)

# Ensure data type is float32
X_3d = X_3d.astype('float32')
Y_3d = Y_3d.astype('float32')

# Train/test split
X_train, X_valid, Y_train, Y_valid = train_test_split(X_3d, Y_3d, test_size=0.2, shuffle=False)

# Check shapes
print(f"X_train shape: {X_train.shape}")  # Should be (num_samples, N_LAGS, N_FEATURES)
print(f"Y_train shape: {Y_train.shape}")  # Should match the output shape
print(f"X_valid shape: {X_valid.shape}")  # Should be (num_samples, N_LAGS, N_FEATURES)
print(f"Y_valid shape: {Y_valid.shape}")  # Should match the output shape

# Initialize the model
model = Sequential()
model.add(LSTM(8, activation='relu', input_shape=(N_LAGS, N_FEATURES)))
model.add(Dropout(0.2))
model.add(RepeatVector(HORIZON))
model.add(LSTM(4, activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(N_FEATURES)))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Fit the model
model.fit(X_train, Y_train, epochs=500, validation_data=(X_valid, Y_valid))

# Make predictions
preds = model.predict(X_valid)


ValueError: cannot reshape array of size 1074 into shape (179,3,1)

In [2]:
# Convert history into inputs and outputs
def to_supervised(train, n_input, n_out=7):
    # Flatten data
    data = train.reshape((train.shape[0] * train.shape[1], train.shape[2]))
    X, y = list(), list()
    in_start = 0
    
    # Step over the entire history one time step at a time
    for _ in range(len(data)):
        # Define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        
        # Ensure we have enough data for this instance
        if out_end < len(data):
            # Prepare input sequence
            x_input = data[in_start:in_end, 0]
            x_input = x_input.reshape((len(x_input), 1))
            X.append(x_input)
            
            # Prepare output sequence
            y.append(data[in_end:out_end, 0])
        
        # Move along one time step
        in_start += 1
    
    return array(X), array(y)


In [ ]:
# univariate multi-step lstm
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM

# split a univariate dataset into train/test sets
def split_dataset(data):
	# split into standard weeks
	train, test = data[1:-328], data[-328:-6]
	# restructure into windows of weekly data
	train = array(split(train, len(train)/7))
	test = array(split(test, len(test)/7))
	return train, test

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
	scores = list()
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
		# store
		scores.append(rmse)
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores

# summarize scores
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))

# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=7):
	# flatten data
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	# step over the entire history one time step at a time
	for _ in range(len(data)):
		# define the end of the input sequence
		in_end = in_start + n_input
		out_end = in_end + n_out
		# ensure we have enough data for this instance
		if out_end <= len(data):
			x_input = data[in_start:in_end, 0]
			x_input = x_input.reshape((len(x_input), 1))
			X.append(x_input)
			y.append(data[in_end:out_end, 0])
		# move along one time step
		in_start += 1
	return array(X), array(y)

# train the model
def build_model(train, n_input):
	# prepare data
	train_x, train_y = to_supervised(train, n_input)
	# define parameters
	verbose, epochs, batch_size = 0, 70, 16
	n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
	# define model
	model = Sequential()
	model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs))
	model.compile(loss='mse', optimizer='adam')
	# fit network
	model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
	return model

# make a forecast
def forecast(model, history, n_input):
	# flatten data
	data = array(history)
	data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
	# retrieve last observations for input data
	input_x = data[-n_input:, 0]
	# reshape into [1, n_input, 1]
	input_x = input_x.reshape((1, len(input_x), 1))
	# forecast the next week
	yhat = model.predict(input_x, verbose=0)
	# we only want the vector forecast
	yhat = yhat[0]
	return yhat

# evaluate a single model
def evaluate_model(train, test, n_input):
	# fit model
	model = build_model(train, n_input)
	# history is a list of weekly data
	history = [x for x in train]
	# walk-forward validation over each week
	predictions = list()
	for i in range(len(test)):
		# predict the week
		yhat_sequence = forecast(model, history, n_input)
		# store the predictions
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next week
		history.append(test[i, :])
	# evaluate predictions days for each week
	predictions = array(predictions)
	score, scores = evaluate_forecasts(test[:, :, 0], predictions)
	return score, scores

# load the new file
dataset = read_csv('household_power_consumption_days.csv', header=0, infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])
# split into train and test
train, test = split_dataset(dataset.values)
# evaluate model and get scores
n_input = 7
score, scores = evaluate_model(train, test, n_input)
# summarize scores
summarize_scores('lstm', score, scores)
# plot scores
days = ['sun', 'mon', 'tue', 'wed', 'thr', 'fri', 'sat']
pyplot.plot(days, scores, marker='o', label='lstm')
pyplot.show()

In [ ]:
# multivariate multi-step encoder-decoder lstm
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

# split a univariate dataset into train/test sets
def split_dataset(data):
	# split into standard weeks
	train, test = data[1:-328], data[-328:-6]
	# restructure into windows of weekly data
	train = array(split(train, len(train)/7))
	test = array(split(test, len(test)/7))
	return train, test

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
	scores = list()
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
		# store
		scores.append(rmse)
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores

# summarize scores
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))

# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=7):
	# flatten data
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	# step over the entire history one time step at a time
	for _ in range(len(data)):
		# define the end of the input sequence
		in_end = in_start + n_input
		out_end = in_end + n_out
		# ensure we have enough data for this instance
		if out_end <= len(data):
			X.append(data[in_start:in_end, :])
			y.append(data[in_end:out_end, 0])
		# move along one time step
		in_start += 1
	return array(X), array(y)

# train the model
def build_model(train, n_input):
	# prepare data
	train_x, train_y = to_supervised(train, n_input)
	# define parameters
	verbose, epochs, batch_size = 0, 50, 16
	n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
	# reshape output into [samples, timesteps, features]
	train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
	# define model
	model = Sequential()
	model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
	model.add(RepeatVector(n_outputs))
	model.add(LSTM(200, activation='relu', return_sequences=True))
	model.add(TimeDistributed(Dense(100, activation='relu')))
	model.add(TimeDistributed(Dense(1)))
	model.compile(loss='mse', optimizer='adam')
	# fit network
	model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
	return model

# make a forecast
def forecast(model, history, n_input):
	# flatten data
	data = array(history)
	data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
	# retrieve last observations for input data
	input_x = data[-n_input:, :]
	# reshape into [1, n_input, n]
	input_x = input_x.reshape((1, input_x.shape[0], input_x.shape[1]))
	# forecast the next week
	yhat = model.predict(input_x, verbose=0)
	# we only want the vector forecast
	yhat = yhat[0]
	return yhat

# evaluate a single model
def evaluate_model(train, test, n_input):
	# fit model
	model = build_model(train, n_input)
	# history is a list of weekly data
	history = [x for x in train]
	# walk-forward validation over each week
	predictions = list()
	for i in range(len(test)):
		# predict the week
		yhat_sequence = forecast(model, history, n_input)
		# store the predictions
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next week
		history.append(test[i, :])
	# evaluate predictions days for each week
	predictions = array(predictions)
	score, scores = evaluate_forecasts(test[:, :, 0], predictions)
	return score, scores

# load the new file
dataset = read_csv('https://raw.githubusercontent.com/sajalsuhane/household_power_consumption/refs/heads/master/household_power_consumption_days.csv', header=0, infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])
# split into train and test
train, test = split_dataset(dataset.values)
# evaluate model and get scores
n_input = 14
score, scores = evaluate_model(train, test, n_input)
# summarize scores
summarize_scores('lstm', score, scores)
# plot scores
days = ['sun', 'mon', 'tue', 'wed', 'thr', 'fri', 'sat']
pyplot.plot(days, scores, marker='o', label='lstm')
pyplot.show()